In [1]:
import os, sys
from pathlib import Path
sys.path.append(str(Path(os.getcwd()).parent))

from transforms import *
from train import *
from utils import *
from models import *
from dataset import *
from learner import *

import torch.optim as optim
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch
import torch.nn as nn
from torchvision.transforms import Normalize

import optuna

In [2]:
EPOCHS = 100
train_dir = os.path.join(Path(os.getcwd()).parent, "data/raw/IRMAS_Training_Data")
valid_dir = os.path.join(Path(os.getcwd()).parent, "data/raw/IRMAS_Validation_Data")
optimizer = optim.Adam

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f'Using: {device}')

Using: cuda


In [3]:
def objective(trial):

    target_sr = 22050
    n_mels = trial.suggest_int("n_mels", 128,224,step=16)
    n_fft = trial.suggest_int("n_fft", 2432, 4992, step=128)
    hop_length = trial.suggest_int("hop_length", 512, 2048, step=128)
    bs = trial.suggest_int("bs", 16, 64, step=16)
    lr = trial.suggest_float("lr", 1e-4, 1e-1, log=True)
    hidden_size = trial.suggest_int("hidden_size", 128,512,step=64)
    num_layers = trial.suggest_int("num_layers", 3,7,step=1)
    weight = trial.suggest_int("pos_weight", 6, 36, step=2)
    drop1 = trial.suggest_float("dropout1", 0.0, 0.5, step=0.1)
    drop2 = trial.suggest_float("dropout2", 0.0, 0.5, step=0.1)
    wd = trial.suggest_float("wd", 1e-8, 0.1, log=True)
    
    preprocess = PreprocessPipelineRNN(target_sr=target_sr)
    transforms = torchvision.transforms.Compose([
        torchaudio.transforms.MelSpectrogram(
            sample_rate=target_sr, 
            f_max=target_sr/2,
            n_mels=n_mels,
            n_fft=n_fft, 
            hop_length=hop_length,
            center=False),
        torchaudio.transforms.AmplitudeToDB(stype="power", top_db=90),
        MinMaxScale(),
        Normalize(0.5,0.5)
    ])

    train_dl = get_loader(train_dir, preprocess=preprocess, augments=None, 
                          transforms=transforms, bs=bs, rnn=True, device=device, 
                          subset="train")
    valid_dl = get_loader(valid_dir, preprocess=preprocess, augments=None,
                          transforms=transforms, bs=bs, rnn=True, 
                          device=device, 
                          subset="valid")
    
    model = RNN(n_mels,hidden_size,num_layers,11, dropout=[drop1, drop2], device=device)

    pos_weight = torch.tensor(weight).repeat(11).float().cuda()
    loss_fn = FocalLoss(pos_weight=pos_weight)

    learn = Learner(train_dl, valid_dl, model, loss_fn, optimizer)
    learn.fit(EPOCHS, lr, wd, verbose=True, no_bar=False)

    return learn.result["f1_score/average"]

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=1)

[I 2023-03-08 13:35:29,479] A new study created in memory with name: no-name-3e0e8a43-2b58-4a7a-bb34-47d0db855cc7


  0%|          | 0/100 [00:00<?, ?it/s]


Epoch: 1
-----------------------------------


  0%|          | 0/105 [00:00<?, ?it/s]

Train loss: 0.262865


...

  0%|          | 0/23 [00:00<?, ?it/s]

Test loss: 0.338040
Global accuracy: 84.29%
Percentage of samples guessed entirely correct: 15.45%
Average f1 score: 0.32
[1.6438024346259115e-09]

Epoch: 21
-----------------------------------


  0%|          | 0/105 [00:00<?, ?it/s]

Train loss: 0.149119


  0%|          | 0/23 [00:00<?, ?it/s]